In [5]:
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models, optimizers
import cv2
import time
from tqdm.auto import tqdm

c:\Users\jasso\.conda\envs\computer-vision-2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
names = ['1 RIBU ASLI', '1 RIBU PALSU', '10 RIBU ASLI', '10 RIBU PALSU', 
         '100 RIBU ASLI', '100 RIBU PALSU', '20 RIBU ASLI', '20 RIBU PALSU', 
         '5 RIBU ASLI', '5 RIBU PALSU', '50 RIBU ASLI', '50 RIBU PALSU']

In [7]:
dataset_path = "C:\Codes\Computer Vision\Project\PROJECT-DATA-UANG-9"

In [8]:
def augment_image(image):
    augmented_images = []

    # Rotasi
    for angle in [-30, 30]:
        rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1.0)
        rotated = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
        augmented_images.append(rotated)

    return augmented_images

In [9]:
def load_dataset_with_folder_structure(folder_path,augment=True):
    images_folder = os.path.join(folder_path, "images")
    labels_folder = os.path.join(folder_path, "labels")
    
    data = []
    labels = []
    count=0
    
    for img_name in os.listdir(images_folder):
        if img_name.endswith(".jpg"): 
            img_path = os.path.join(images_folder, img_name)
            label_path = os.path.join(labels_folder, os.path.splitext(img_name)[0] + ".txt")  # File label

            img = cv2.imread(img_path)
            if img is not None and os.path.exists(label_path):  # Pastikan gambar dan label ada
                resized_img = cv2.resize(img, (512, 512))  # Resize gambar
                data.append(resized_img)
                
                with open(label_path, "r") as f:
                    first_line = f.readline().strip()  # Ambil baris pertama
                    class_index = int(first_line.split()[0])  # Digit pertama sebagai label
                    labels.append(class_index)

                if augment:
                    augmented_imgs = augment_image(resized_img)
                    for aug_img in augmented_imgs:
                        data.append(aug_img)
                        labels.append(class_index)
    
    
    return np.asarray(data, dtype=np.uint8), np.array(labels, dtype=np.int32)

In [10]:
x_val, y_val=load_dataset_with_folder_structure(dataset_path+'/valid/', False)
x_test, y_test=load_dataset_with_folder_structure(dataset_path+'/test/', False)
x_train, y_train=load_dataset_with_folder_structure(dataset_path+'/train/', False)

In [11]:
import torch
from torchvision import models

In [12]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = torch.nn.Linear(in_features=2048, out_features=12)
        # self.resnet.compile()
        
    def forward(self, x):
        return self.resnet(x)

In [13]:
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [14]:
class MoneyDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        image = self.x[idx]
        label = self.y[idx]

        if self.transform:
            image = self.transform(image)
            label = torch.tensor(label)

        return image, label

In [15]:
transform = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = MoneyDataset(x_train, y_train, transform)
val_dataset = MoneyDataset(x_val, y_val, transform)
test_dataset = MoneyDataset(x_test, y_test, transform)

In [16]:
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [28]:
test_loader = DataLoader(test_dataset)

In [17]:
len(train_loader)

479

In [18]:

def run_one_epoch(
    epoch: int,
    ds_sizes,
    dataloaders,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    loss: nn.Module,
    scheduler: torch.optim.lr_scheduler,
    criterion: nn.Module
):
    global best_AUROC
    
    metrics = {}
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    for phase in ["train", "val"]:

        if phase == "train":
            model.train()
        else:
            model.eval()
        
        avg_loss = 0

        for images, labels in tqdm(dataloaders[phase], total=len(dataloaders[phase])):
            labels = labels.type(torch.LongTensor)
            images = images.to(device)
            labels = labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Track history if in phase == "train"
            with torch.set_grad_enabled(phase == "train"):
                outputs = model(images)
                
                # print(outputs)
                # print(labels)
                loss = criterion(outputs, labels)
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()

            avg_loss += loss.item() * images.size(0)

        epoch_loss = avg_loss / ds_sizes[phase]
        
        # step the scheduler
        if phase == "train":
            scheduler.step(epoch_loss)
        

        # Metrics tracking
        if phase == "train":
            metrics["train_loss"] = round(epoch_loss, 3)
        else:
            metrics["val_loss"] = round(epoch_loss, 3)

    return metrics

In [19]:
def train(dataloaders, ds_sizes, model, optimizer, criterion, scheduler):
    for epoch in range(20):
        start = time.time()

        metrics = run_one_epoch(
            epoch=epoch,
            ds_sizes=ds_sizes,
            dataloaders=dataloaders,
            model=model,
            optimizer=optimizer,
            loss=criterion,
            scheduler=scheduler,
            criterion=criterion
        )

        end = time.time() - start

        print(f"Epoch {epoch+1} completed in: {round(end/60, 3)} mins, val_loss: {metrics['val_loss']}, train_loss: {metrics['train_loss']}")
    return model

In [20]:
model = ResNet50().to('cuda')
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

dataloaders = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader
}

ds_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset),
    "test": len(test_dataset)
}


train(dataloaders, ds_sizes, model, optimizer, criterion, scheduler)

c:\Users\jasso\.conda\envs\computer-vision-2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jasso\.conda\envs\computer-vision-2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 69/69 [00:05<00:00, 11.57it/s]


Epoch 1 completed in: 2.891 mins, val_loss: 0.14, train_loss: 0.804


100%|██████████| 69/69 [00:05<00:00, 11.62it/s]


Epoch 2 completed in: 2.888 mins, val_loss: 0.027, train_loss: 0.257


100%|██████████| 69/69 [00:06<00:00, 10.74it/s]


...

100%|██████████| 69/69 [00:05<00:00, 11.54it/s]


Epoch 19 completed in: 2.882 mins, val_loss: 0.017, train_loss: 0.013


100%|██████████| 69/69 [00:05<00:00, 11.79it/s]

Epoch 20 completed in: 2.879 mins, val_loss: 0.017, train_loss: 0.011


ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [21]:
model.eval()

ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [ ]:
#predict
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f"predicted: {predicted}, actual: {labels}")

print(correct)
print(total)
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.int32)
predicted: tensor([4], device=

: 

In [ ]:
torch.save(model.state_dict(), 'model.pth')